<a href="https://colab.research.google.com/github/prawizard/TweetsClassification_NLP/blob/main/TweetEval/Sentiment_Analysis_NLP_Progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
import requests
import collections

In [2]:

import os, string, collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import snowballstemmer
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, load_model
from keras.layers.embeddings import Embedding
from keras.layers import Flatten, Dense, Dropout, Convolution1D, MaxPooling1D, SpatialDropout1D, Input 
from keras.layers import GlobalMaxPooling1D, concatenate, LSTM, Bidirectional
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
import numpy as np
from numpy import array
import pandas as pd
from sklearn.model_selection import train_test_split

# Constants

In [4]:
TRAIN_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt"
TRAIN_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt"
VAL_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_text.txt"
VAL_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_labels.txt"
TEST_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt"
TEST_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt"
VOCAB_SIZE=2000

# Access Data from the Files

## Download the .txt files to our runtime

In [5]:
r = requests.get(TRAIN_TEXT_URL, allow_redirects=True)
open('train_text.txt', 'wb').write(r.content)

r = requests.get(TRAIN_LABELS_URL, allow_redirects=True)
open('train_labels.txt', 'wb').write(r.content)

r = requests.get(VAL_TEXT_URL, allow_redirects=True)
open('val_text.txt', 'wb').write(r.content)

r = requests.get(VAL_LABELS_URL, allow_redirects=True)
open('val_labels.txt', 'wb').write(r.content)

r = requests.get(TEST_TEXT_URL, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r = requests.get(TEST_LABELS_URL, allow_redirects=True)
open('test_labels.txt', 'wb').write(r.content)

1720

## Read data from the files downloaded

In [6]:
# Tweets
stream=open("train_text.txt")
tweets=stream.readlines()
stream.close()
val_stream=open("val_text.txt")
val_tweets=val_stream.readlines()
val_stream.close()
test_stream=open("test_text.txt")
test_tweets=test_stream.readlines()
test_stream.close()

# Labels
stream=open("train_labels.txt")
tweetsLabels=stream.readlines()
stream.close()
val_stream=open("val_labels.txt")
val_tweetsLabels=val_stream.readlines()
val_stream.close()
test_stream=open("test_labels.txt")
test_tweetsLabels=test_stream.readlines()
test_stream.close()


# Labels
labels=[0]*len(tweetsLabels)
for i in range(len(tweetsLabels)):
  if tweetsLabels[i].find('\n')!=-1:
    labels[i]=int(re.sub('\n', '', tweetsLabels[i]))
val_labels=[0]*len(val_tweetsLabels)
for i in range(len(val_tweetsLabels)):
  if val_tweetsLabels[i].find('\n')!=-1:
    val_labels[i]=int(re.sub('\n', '', val_tweetsLabels[i]))
test_labels=[0]*len(test_tweetsLabels)
for i in range(len(test_tweetsLabels)):
  if test_tweetsLabels[i].find('\n')!=-1:
    test_labels[i]=int(re.sub('\n', '', test_tweetsLabels[i]))


In [7]:
print('Samples in Training set : ',len(labels),', Validation set : ', len(val_labels),', Test set : ', len(test_labels))

Samples in Training set :  11916 , Validation set :  1324 , Test set :  860


# Pre-processing of the text in tweets

## Remove the '@User' text parts from tweets


In [8]:

for i in range(len(tweets)):
  if tweets[i].find('@user')!=-1:
    tweets[i]=re.sub('@user', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('@user')!=-1:
    val_tweets[i]=re.sub('@user', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('@user')!=-1:
    test_tweets[i]=re.sub('@user', '', test_tweets[i])

## Remove hashtags

In [9]:
for i in range(len(tweets)):
  if tweets[i].find('#[a-zA-Z]+')!=-1:
    tweets[i]=re.sub('#[a-zA-Z]+', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('#[a-zA-Z]+')!=-1:
    val_tweets[i]=re.sub('#[a-zA-Z]+', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('#[a-zA-Z]+')!=-1:
    test_tweets[i]=re.sub('#[a-zA-Z]+', '', test_tweets[i])

## Remove escape characters, unnecessary characters if present and correct wrongly spelt words

In [10]:
for i in range(len(tweets)):
  if tweets[i].find('\n')!=-1:
    tweets[i]=re.sub('\n', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('\n')!=-1:
    val_tweets[i]=re.sub('\n', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('\n')!=-1:
    test_tweets[i]=re.sub('\n', '', test_tweets[i])

# Unnecessary dots
p='\.\.\.|\.\.'

for i in range(len(tweets)):
  tweets[i]=re.sub(p, '', tweets[i])
  tweets[i]=tweets[i].lower()

for i in range(len(val_tweets)):
  val_tweets[i]=re.sub(p, '', val_tweets[i])
  val_tweets[i]=val_tweets[i].lower()

for i in range(len(test_tweets)):
  test_tweets[i]=re.sub(p, '', test_tweets[i])
  test_tweets[i]=test_tweets[i].lower()

In [15]:
rows=[]
rowIndices=[]
for i in range(len(tweets)):
  rows.append({"TWEET":tweets[i], "CATEGORY":labels[i]})
  rowIndices.append(i+1)
df=pd.DataFrame(rows, index=rowIndices)

val_rows=[]
val_rowIndices=[]
for i in range(len(val_tweets)):
  val_rows.append({"TWEET":val_tweets[i], "CATEGORY":val_labels[i]})
  val_rowIndices.append(i+1)
val_df=pd.DataFrame(val_rows, index=val_rowIndices)

test_rows=[]
test_rowIndices=[]
for i in range(len(test_tweets)):
  test_rows.append({"TWEET":test_tweets[i], "CATEGORY":test_labels[i]})
  test_rowIndices.append(i+1)
test_df=pd.DataFrame(test_rows, index=test_rowIndices)

In [30]:
frames=[df, val_df, test_df]
df=pd.concat(frames)

In [31]:
type(df)

pandas.core.frame.DataFrame

In [32]:
df

,TWEET,CATEGORY
1,bono who cares. soon people will understand t...,0
2,eight years the republicans denied obama’s pi...,1
3,get him some line help. he is gonna be just f...,0
4,she is great. hi fiona!,0
5,she has become a parody unto herself? she has...,1
...,...,...
856,#cnn irrationally argues 4 legalising #abortio...,0
857,"city of chicago, democrat run wi...",0
858,#conservatives don’t care what you postit’s p...,1
859,#antifa #resist trump is trying to bring world...,0


In [33]:
df['CATEGORY'].value_counts()

0    9460
1    4640
Name: CATEGORY, dtype: int64

In [34]:
text = df['TWEET'].tolist()
text[:10]

[' bono who cares. soon people will understand that they gain nothing from following a phony celebrity. become a leader of your people instead or help and support your fellow countrymen. ',
 ' eight years the republicans denied obama’s picks. breitbarters outrage is as phony as their fake president. ',
 ' get him some line help. he is gonna be just fine. as the game went on you could see him progressing more with his reads. he brought what has been missing. the deep ball presence. now he just needs a little more time ',
 '  she is great. hi fiona! ',
 " she has become a parody unto herself? she has certainly taken some heat for being such an.well idiot. could be optic too  who know with liberals  they're all optics.  no substance ",
 '               this is the vetsresistsquadron"" is bullshit they are girl scout veterans, i have never met any other veterans or served with anyone that was a gun control advocate? have you?"" ',
 ' your looking more like a plant #maga #walkaway ',
 '  lo

In [35]:
y = df['CATEGORY']

In [36]:
token = Tokenizer()
token.fit_on_texts(text)
token

In [37]:
vocab_size = len(token.word_index) + 1
vocab_size

23433

In [38]:
import itertools 
print(dict(itertools.islice(token.index_word.items(), 100)))

{1: 'the', 2: 'is', 3: 'to', 4: 'a', 5: 'and', 6: 'you', 7: 'of', 8: 'are', 9: 'i', 10: 'he', 11: 'in', 12: 'that', 13: 'for', 14: 'she', 15: 'it', 16: 'this', 17: 'on', 18: 'not', 19: 'with', 20: 'they', 21: 'have', 22: 'be', 23: 'liberals', 24: 'gun', 25: 'so', 26: 'all', 27: 'control', 28: 'antifa', 29: 'your', 30: 'like', 31: 'what', 32: 'as', 33: 'but', 34: 'we', 35: 'just', 36: 'about', 37: 'her', 38: 'maga', 39: 'was', 40: 'if', 41: 'conservatives', 42: 'will', 43: 'do', 44: 'who', 45: 'people', 46: 'no', 47: 'his', 48: 'at', 49: 'or', 50: 'my', 51: 'by', 52: 'has', 53: 'from', 54: 'how', 55: 'an', 56: 'out', 57: 'up', 58: 'their', 59: 'me', 60: 'get', 61: 'one', 62: 'amp', 63: 'know', 64: 'trump', 65: 'why', 66: 'when', 67: 'more', 68: 'because', 69: 'can', 70: 'him', 71: 'now', 72: 'them', 73: 'think', 74: "don't", 75: 'there', 76: 'would', 77: 'should', 78: 'right', 79: 'our', 80: 'good', 81: 'only', 82: 'us', 83: "it's", 84: 'want', 85: 'time', 86: 'go', 87: 'see', 88: 'goin

In [39]:
x = ['i to the a and']
token.texts_to_sequences(x)

[[9, 3, 1, 4, 5]]

In [40]:
encoded_text = token.texts_to_sequences(text)
print(encoded_text[:30])

[[1370, 44, 694, 532, 45, 42, 324, 12, 20, 1923, 145, 53, 542, 4, 2279, 3978, 437, 4, 660, 7, 29, 45, 480, 49, 182, 5, 159, 29, 1638, 10016], [3979, 147, 1, 263, 4601, 2875, 3980, 10017, 1566, 2, 32, 2279, 32, 58, 285, 135], [60, 70, 107, 543, 182, 10, 2, 415, 22, 35, 517, 32, 1, 298, 544, 17, 6, 154, 87, 70, 10018, 67, 19, 47, 10019, 10, 1125, 31, 52, 93, 1371, 1, 569, 2021, 3501, 71, 10, 35, 224, 4, 287, 67, 85], [14, 2, 138, 1208, 3981], [14, 52, 437, 4, 6990, 10020, 746, 14, 52, 1372, 850, 107, 5520, 13, 94, 188, 55, 117, 570, 154, 22, 10021, 102, 44, 63, 19, 23, 361, 26, 5521, 46, 6991], [16, 2, 1, 10022, 2, 533, 20, 8, 362, 10023, 2022, 9, 21, 90, 1080, 101, 118, 2022, 49, 2639, 19, 213, 12, 39, 4, 24, 27, 1924, 21, 6], [29, 288, 67, 30, 4, 2876, 38, 352], [161, 586, 260, 1, 119, 1253, 135, 11, 79, 10024, 260, 6992, 119, 7, 1, 2133, 10025, 105, 5, 639, 174, 17, 1, 78, 2431, 158, 38], [93, 4, 3127, 649, 220, 204, 119, 7, 6, 110, 1126, 97, 12, 10, 2, 1081, 4, 1209, 19, 951, 368, 18

In [41]:
ll=len(encoded_text[0])
for tw in encoded_text:
  l=len(tw)
  if l>ll:
    ll=l
ll

62

In [42]:
max_length = 65
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')
print(X)

[[1370   44  694 ...    0    0    0]
 [3979  147    1 ...    0    0    0]
 [  60   70  107 ...    0    0    0]
 ...
 [  41   95  184 ...    0    0    0]
 [  28  888   64 ...    0    0    0]
 [9691    6   89 ...    0    0    0]]


In [43]:
X.shape

(14100, 65)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [59]:
vec_size = 350

model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length))

model.add(Conv1D(64, 8, activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.3))

model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation='sigmoid'))

In [60]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [61]:
%%time
model.fit(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Epoch 1/5
353/353 [==============================] - 30s 82ms/step - loss: 0.6366 - accuracy: 0.6600 - val_loss: 0.5828 - val_accuracy: 0.7348
Epoch 2/5
353/353 [==============================] - 29s 81ms/step - loss: 0.4771 - accuracy: 0.7912 - val_loss: 0.5584 - val_accuracy: 0.7408
Epoch 3/5
353/353 [==============================] - 29s 81ms/step - loss: 0.3088 - accuracy: 0.8837 - val_loss: 0.5559 - val_accuracy: 0.7355
Epoch 4/5
353/353 [==============================] - 29s 82ms/step - loss: 0.1933 - accuracy: 0.9328 - val_loss: 0.6302 - val_accuracy: 0.7191
Epoch 5/5
353/353 [==============================] - 29s 81ms/step - loss: 0.1155 - accuracy: 0.9630 - val_loss: 0.7487 - val_accuracy: 0.7262
CPU times: user 4min 4s, sys: 6.99 s, total: 4min 11s
Wall time: 2min 24s
